In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot
import seaborn as sns
from os import cpu_count as ncpu

In [2]:
data = pd.read_csv(filepath_or_buffer='./data_normalized.csv',sep=',')
data = data.drop(labels=['Unnamed: 0','MeanScore','MeanScoreDisc'],axis=1)

In [3]:
cols = list(data.columns.values)
cols_target = list(cols[1:4])
cols_target_disc = list(cols[-3:])
cols_var = list(data.drop(labels=cols_target+cols_target_disc,axis=1).columns.values)

# <span style="color:rgb(0,200,200)"> Selección de variables con sklearn</span>

In [8]:
training = data[cols_var]
target = data[cols_target_disc]
display([[training.columns.values],[],[target.columns.values]])
n_classes = 3

[[array(['NrSiblings', 'LunchType', 'WklyStudyHours', 'ParentEduc',
         'TestPrep', 'Gender', 'EthnicGroup', 'ParentMaritalStatus',
         'PracticeSport', 'IsFirstChild', 'TransportMeans'], dtype=object)],
 [],
 [array(['MathScoreDisc', 'ReadingScoreDisc', 'WritingScoreDisc'],
        dtype=object)]]

## <span style="color:rgb(200,0,0)"> Select Kbest </span>

In [9]:
from sklearn.feature_selection import chi2,f_classif, SelectKBest

In [11]:
crit = [f_classif]

for targ in cols_target_disc:
    print(f"\n--- {targ} ---")

    x,y = [np.array(training),(np.array(target[targ]))]
    
    result = []        
    for method in crit:
        selector = SelectKBest(score_func=method,k=n_classes,)
        atributes = selector.fit(x,y).get_support()
        atributos = [cols_var[i] for i in list(atributes.nonzero()[0])]
        result.append(atributos)
        
    result = pd.DataFrame(dict(zip(crit,result)),columns=crit)
    display(result)



--- MathScoreDisc ---


,<function f_classif at 0x7fc3a52d1800>
0,LunchType
1,ParentEduc
2,EthnicGroup



--- ReadingScoreDisc ---


,<function f_classif at 0x7fc3a52d1800>
0,LunchType
1,TestPrep
2,Gender



--- WritingScoreDisc ---


,<function f_classif at 0x7fc3a52d1800>
0,LunchType
1,TestPrep
2,Gender


## <span style="color:rgb(200,0,0)">Árboles de desición</span>

In [12]:
from sklearn.feature_selection import RFECV,RFE
from sklearn.ensemble import ExtraTreesRegressor

In [14]:
crit = ['squared_error', 'absolute_error', 'friedman_mse', 'poisson']
crit = ['friedman_mse', 'poisson']
for targ in cols_target_disc:
    print(f"\n--- {targ} ---")

    x,y = [np.array(training),np.array(target[targ])]
    result = []        
    for method in crit:
        model = ExtraTreesRegressor(criterion=method,n_jobs=ncpu())
        selector = RFE(estimator=model,n_features_to_select=n_classes)
        atributes = selector.fit(x,y).support_
        atributos = [cols_var[i] for i in list(atributes.nonzero()[0])]
        result.append(atributos)
        
    result = pd.DataFrame(dict(zip(crit,result)),columns=crit)
    display(result)



--- MathScoreDisc ---


,friedman_mse,poisson
0,LunchType,LunchType
1,ParentEduc,ParentEduc
2,EthnicGroup,EthnicGroup



--- ReadingScoreDisc ---


,friedman_mse,poisson
0,LunchType,LunchType
1,ParentEduc,ParentEduc
2,Gender,Gender



--- WritingScoreDisc ---


,friedman_mse,poisson
0,LunchType,LunchType
1,TestPrep,TestPrep
2,Gender,Gender


In [35]:
cols_selected_math = ['LunchType','ParentEduc','EthnicGroup'] # 3 características
cols_selected_writing = ['LunchType','TestPrep','ParentEduc','Gender'] # 4 características
cols_selected_reading = ['LunchType','TestPrep','Gender'] # 3 características
cols_selected = [cols_selected_math,cols_selected_reading,cols_selected_writing]

# <span style="color:rgb(0,200,200)">Modelos</span>

In [34]:
from sklearn.model_selection import train_test_split as split

## <span style="color:rgb(200,0,200)">Regresión lineal</span>

In [50]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error,r2_score, confusion_matrix

In [53]:
for col,targ in enumerate(cols_target_disc):
    print(f"--- {targ} ---")
    x = training[cols_selected[col]]
    print(f"training with features: {cols_selected[col]}")

    x_train,x_test,y_train,y_test = split(x,target[targ],train_size=.7)

    regresor = LinearRegression(n_jobs=ncpu())
    result = regresor.fit(x_train,y_train)
    y_predicted = result.predict(x_test)
    display(confusion_matrix(y_true=y_test,y_pred=y_predicted))
    
    

    

--- MathScoreDisc ---
training with features: ['LunchType', 'ParentEduc', 'EthnicGroup']


ValueError: continuous is not supported

In [44]:
result.predict(x_test)

array([0.70991895, 0.81704766, 0.5172194 , ..., 0.62811826, 0.62141238,
       0.60085137])

## <span style="color:rgb(200,0,200)">Regresión logísitica</span>

## <span style="color:rgb(200,0,200)">K-Vecinos</span>

## <span style="color:rgb(200,0,200)">Árboles de desición</span>